In [ ]:
import os
import re


pars=[]
path='/content/drive/MyDrive/NLP/NLP_Project/'


with open(os.path.join(path+'wonderland_edited.txt'), 'r') as a:
        word = re.sub(r'someword=|\,.*|\#.*','', a.read())
        word = re.sub(r'\n+', '\n', word).strip()
        pars.append(word)

for k in pars:
   print(k)

﻿CHAPTER I. Down the Rabbit-Hole
Alice was beginning to get very tired of sitting by her sister on the
bank
book her sister was reading
it
conversations?'
So she was considering in her own mind (as well as she could
hot day made her feel very sleepy and stupid)
of making a daisy-chain would be worth the trouble of getting up and
picking the daisies
close by her.
There was nothing so VERY remarkable in that; nor did Alice think it so
VERY much out of the way to hear the Rabbit say to itself
Oh dear! I shall be late!' (when she thought it over afterwards
occurred to her that she ought to have wondered at this
it all seemed quite natural); but when the Rabbit actually TOOK A WATCH
OUT OF ITS WAISTCOAT-POCKET
Alice started to her feet
never before seen a rabbit with either a waistcoat-pocket
to take out of it
after it
rabbit-hole under the hedge.
In another moment down went Alice after it
in the world she was to get out again.
The rabbit-hole went straight on like a tunnel for some way
dip

In [ ]:
len(pars)

1

In [ ]:
# with open(os.path.join(path,"output.txt"), "w") as txt_file:
#     for line in pars:
#         txt_file.write(" ".join(line) + "\n")

In [ ]:
with open(os.path.join(path,"outpu.txt"), "w") as txt_file:
            txt_file.write(" ".join(pars))

In [ ]:
# https://medium.com/analytics-vidhya/nlp-word-prediction-by-using-bidirectional-lstm-9c01c24b2725

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

In [ ]:
# We read data and specified Tokenizer
tokenizer = Tokenizer()
data = open(path+"outpu.txt").read()
# We splited the data by whitespaces and created sentence list 
corpus = data.lower().split("\n")
# with "fit_on_text" method tokenized each sentence in the corpus
tokenizer.fit_on_texts(corpus)
# Afterwards we created word index specified unique number of words in the corpus by assign index number each of them
# like this; {'car': 1, 'prison': 2, 'him': 3, 'welcome': 4}
total_words = len(tokenizer.word_index) + 1 
# we'll add one to this to consider vocabulary words
# This is a key value pair with the key being the word and the value being the token for that word

print(tokenizer.word_index)
print(total_words)

{'the': 1, "'": 2, 'to': 3, 'a': 4, 'of': 5, 'it': 6, 'and': 7, 'she': 8, 'was': 9, 'in': 10, 'alice': 11, 'you': 12, 'her': 13, 'that': 14, 'as': 15, 'said': 16, 'i': 17, 'at': 18, 'on': 19, 'be': 20, 'all': 21, 'with': 22, "'i": 23, 'had': 24, 'for': 25, 'little': 26, 'so': 27, 'this': 28, 'out': 29, 'very': 30, 'not': 31, 'is': 32, 'about': 33, 'down': 34, 'they': 35, 'what': 36, 'up': 37, 'like': 38, 'them': 39, 'he': 40, 'his': 41, 'one': 42, 'herself': 43, 'time': 44, 'again': 45, 'but': 46, 'have': 47, 'would': 48, 'were': 49, 'into': 50, 'could': 51, 'when': 52, 'did': 53, 'me': 54, 'queen': 55, 'there': 56, 'off': 57, 'no': 58, 'then': 59, 'say': 60, 'do': 61, 'thing': 62, 'much': 63, 'see': 64, 'thought': 65, 'first': 66, 'their': 67, 'your': 68, 'know': 69, 'way': 70, 'think': 71, 'its': 72, 'went': 73, 'or': 74, 'are': 75, 'rabbit': 76, 'if': 77, 'my': 78, 'more': 79, "'you": 80, 'get': 81, 'some': 82, 'began': 83, 'an': 84, 'head': 85, 'now': 86, "don't": 87, 'by': 88, 'qu

In [ ]:
# example: In the town of Athy one Jeremy Lanigan
                      
#                  [4,2,66,67,68,69,70]
# This will convert a line of text like,sentence above into a list of tokens representing the words.
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
## finding the maximum length of sequence present in the corpus
max_sequence_len = max([len(x) for x in input_sequences])
print(max_sequence_len)

18


In [ ]:
## pad sequences
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
##create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [ ]:
print(xs)

[[   0    0    0 ...    0    0 1211]
 [   0    0    0 ...    0 1211   17]
 [   0    0    0 ... 1211   17   34]
 ...
 [   0    0    0 ...    0 2284   13]
 [   0    0    0 ... 2284   13  295]
 [   0    0    0 ...   13  295  308]]


In [ ]:
print(ys)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
## Model Build
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(xs, ys, epochs=100, verbose=1)
#print model.summary()
print(model)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
422/422 [==============================] - 16s 21ms/step - loss: 6.0681 - accuracy: 0.0774
Epoch 2/100
422/422 [==============================] - 9s 21ms/step - loss: 5.0959 - accuracy: 0.1357
Epoch 3/100
422/422 [==============================] - 9s 21ms/step - loss: 4.3628 - accuracy: 0.1833
Epoch 4/100
422/422 [==============================] - 9s 21ms/step - loss: 3.7139 - accuracy: 0.2344
Epoch 5/100
422/422 [==============================] - 9s 21ms/step - loss: 3.1520 - accuracy: 0.3003
Epoch 6/100
422/422 [==============================] - 9s 21ms/step - loss: 2.6515 - accuracy: 0.3754
Epoch 7/100
422/422 [==============================] - 9s 21ms/step - loss: 2.2570 - accuracy: 0.4432
Epoch 8/100
422/422 [==============================] - 9s 21ms/step - loss: 2.0134 - accuracy: 0.4922
Epoch 9/100
422/422 [==============================] - 9s 21ms/step - loss: 1.8036 - accuracy: 0.5367
Epoch 10/100
422/422 [==============================] - 9s 21ms/step - loss: 1.61

In [ ]:
model.save(path)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/NLP/NLP_Project/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/NLP/NLP_Project/assets


In [ ]:
## seed text and next words
seed_text = "Alice had been"
next_words = 1

In [ ]:
token_list = tokenizer.texts_to_sequences([seed_text])[0]
token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
predicted = model.predict(token_list, verbose=0)

In [ ]:
 print(tokenizer.texts_to_sequences([seed_text])[0])

[11, 24, 130, 201]


array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         11,  24, 130, 201]], dtype=int32)

In [ ]:
predicted

array([[1.3530761e-16, 1.2004097e-07, 4.0093082e-10, ..., 2.0292965e-22,
        6.2296903e-18, 1.1049933e-16]], dtype=float32)

In [ ]:
##Prediction
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    output_word = ""
    # print(predicted)
    
    for word, index in tokenizer.word_index.items():
        print(word,index)
        print(len(predicted[0]))
        
        if(index in predicted[0]):
            output_word = word
            
seed_text += " " + output_word
print(seed_text)

the 1
2285
' 2
2285
to 3
2285
a 4
2285
of 5
2285
it 6
2285
and 7
2285
she 8
2285
was 9
2285
in 10
2285
alice 11
2285
you 12
2285
her 13
2285
that 14
2285
as 15
2285
said 16
2285
i 17
2285
at 18
2285
on 19
2285
be 20
2285
all 21
2285
with 22
2285
'i 23
2285
had 24
2285
for 25
2285
little 26
2285
so 27
2285
this 28
2285
out 29
2285
very 30
2285
not 31
2285
is 32
2285
about 33
2285
down 34
2285
they 35
2285
what 36
2285
up 37
2285
like 38
2285
them 39
2285
he 40
2285
his 41
2285
one 42
2285
herself 43
2285
time 44
2285
again 45
2285
but 46
2285
have 47
2285
would 48
2285
were 49
2285
into 50
2285
could 51
2285
when 52
2285
did 53
2285
me 54
2285
queen 55
2285
there 56
2285
off 57
2285
no 58
2285
then 59
2285
say 60
2285
do 61
2285
thing 62
2285
much 63
2285
see 64
2285
thought 65
2285
first 66
2285
their 67
2285
your 68
2285
know 69
2285
way 70
2285
think 71
2285
its 72
2285
went 73
2285
or 74
2285
are 75
2285
rabbit 76
2285
if 77
2285
my 78
2285
more 79
2285
'you 80
2285
get 81
2285
some